# Q1. Refactoring

Now we need to create the "main" block from which we'll invoke the main function. How does the if statement that we use for this looks like?

`if __name__ == "__main__":`

# Q2. Installing pytest

Next, create a folder tests and create two files. One will be the file with tests. We can name it test_batch.py.

What should be the other file?

 `__init__.py`

# Q3. Writing first unit test

How many rows should be there in the expected dataframe?

* 1
* 2
* 3
* 4 ✅

# Q4. Mocking S3 with Localstack

* --backend-store-uri
* --profile
* --endpoint-url ✅
* --version